In [ ]:
!pip install boto3
!pip install langchain_aws
!pip install langchain_core

In [12]:
# Check pydantic verssion
!pip show pydantic
# Downgrading pydantic to 1.10.12
!pip install pydantic==1.10.12 --force-reinstall

Name: pydantic
Version: 2.12.0
Summary: Data validation using Python type hints
Home-page: https://github.com/pydantic/pydantic
Author: 
Author-email: Samuel Colvin <s@muelcolvin.com>, Eric Jolibois <em.jolibois@gmail.com>, Hasan Ramezani <hasan.r67@gmail.com>, Adrian Garcia Badaracco <1755071+adriangb@users.noreply.github.com>, Terrence Dorsey <terry@pydantic.dev>, David Montague <david@pydantic.dev>, Serge Matveenko <lig@countzero.co>, Marcelo Trylesinski <marcelotryle@gmail.com>, Sydney Runkle <sydneymarierunkle@gmail.com>, David Hewitt <mail@davidhewitt.io>, Alex Hall <alex.mojaki@gmail.com>, Victorien Plot <contact@vctrn.dev>, Douwe Maan <hi@douwe.me>
License: 
Location: d:\thesis\thesis\.venv\lib\site-packages
Requires: annotated-types, pydantic-core, typing-extensions, typing-inspection
Required-by: langchain-aws, langchain-core, langsmith


In [1]:
import boto3
import json
from typing import List
import time
import random
from botocore.exceptions import ClientError

In [4]:
import boto3, json, time, random
from botocore.exceptions import ClientError

class BedrockAPICalls:
    def __init__(self, secrets_path='secrets.json', region='us-west-2',
                 model_id='anthropic.claude-3-sonnet-20240229-v1:0'):
        # Load secrets (optional if using IAM roles)
        with open(secrets_path, 'r') as file:
            secrets = json.load(file)
            self.aws_access_key_id = secrets.get('AWS_ACCESS_KEY')
            self.aws_secret_access_key = secrets.get('AWS_SECRET_ACCESS_KEY')

        session = boto3.Session(
            aws_access_key_id=self.aws_access_key_id,
            aws_secret_access_key=self.aws_secret_access_key,
            region_name=region
        )

        self.model_id = model_id
        self.client = session.client("bedrock-runtime")

        # Rate limiting
        self.max_rpm = 60   # adjust to your quota
        self.seconds_per_request = 60 / self.max_rpm
        self._last_call_time = 0

    def call_llm(self, prompt: str) -> str:
        # Pacing
        now = time.time()
        elapsed = now - self._last_call_time
        if elapsed < self.seconds_per_request:
            time.sleep(self.seconds_per_request - elapsed)

        retry_count = 0
        while retry_count < 5:
            try:

                response = self.client.invoke_model(
                    modelId=self.model_id,
                    body=json.dumps({
                        # Anthropic
                        # "anthropic_version": "bedrock-2023-05-31",
                        # "messages": [{"role": "user", 
                        #               "content": prompt}],
                        # "max_tokens": 10,
                        # "temperature": 0.1

                        # Llama 3.1
                        # "prompt": prompt,
                        # "max_gen_len": 5,
                        # "temperature": 0.1

                        # Mistral
                        "prompt": prompt,
                        "max_tokens": 10,
                        "temperature": 0.1
                    }),
                    contentType="application/json",
                    accept="application/json"
                )
                self._last_call_time = time.time()
                result = json.loads(response["body"].read())
                return result #['content'][0]['text'] #['generation'] #
            except ClientError as e:
                if e.response['Error']['Code'] in ("ThrottlingException", "TooManyRequestsException"):
                    wait_time = random.uniform(5, 15)
                    print(f"Throttled. Waiting {wait_time:.1f}s...")
                    time.sleep(wait_time)
                    retry_count += 1
                else:
                    raise
        raise Exception("Max retries exceeded for Bedrock API calls.")


In [5]:
item = BedrockAPICalls(model_id="mistral.mistral-large-2407-v1:0")

prompt =  "How are you?"
result = item.call_llm(prompt)
print(result)

result

{'id': '098400c6-725b-4c8d-82bb-010ec19447d5', 'object': 'chat.completion', 'created': 1761147417, 'model': 'mistral-large-2407', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '\n\nI’mgood. I’m', 'tool_calls': None, 'index': None, 'tool_call_id': None}, 'finish_reason': 'length', 'logprobs': None, 'context_logits': None, 'generation_logits': None}], 'usage': {'prompt_tokens': 5, 'total_tokens': 15, 'completion_tokens': 10}}


{'id': '098400c6-725b-4c8d-82bb-010ec19447d5',
 'object': 'chat.completion',
 'created': 1761147417,
 'model': 'mistral-large-2407',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '\n\nI’mgood. I’m',
    'tool_calls': None,
    'index': None,
    'tool_call_id': None},
   'finish_reason': 'length',
   'logprobs': None,
   'context_logits': None,
   'generation_logits': None}],
 'usage': {'prompt_tokens': 5, 'total_tokens': 15, 'completion_tokens': 10}}

In [6]:
result['choices'][0]['message']['content']

'\n\nI’mgood. I’m'